In [6]:
pip install tensorflow


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/1c/c3/17c6aa1dd5bc8cea5bf00d0c3a021a5dd1680c250861cc877a7e556e4b9b/tensorflow-2.14.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.14.0 from https://files.pythonhosted.org/packages/cb/90/599c79a248dcae6935331113649de5d75427e320efde21b583648b498584/tensorflow_intel-2.14.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.m

In [7]:
#Setup Data
apikey  = 'QHOA07WHc6ZkNS2A9i4bTcTbYuOpEqLI3eHu6iWp02gDOnmwMFpQsSQDO4nytISY'
secret = 'yvHF7zGHB9CVj8cSHQb0j8cZdjhfNSdJ7FcWJuvIfVlNcCh4aT6UneTJRjcokzH9'

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import warnings
from termcolor import colored
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
    
warnings.filterwarnings('ignore')

#Autenticate
client = Client(apikey, secret)
print("Client initialized successfully")

Client initialized successfully


In [15]:
#Chose to load a Historical Data
def seconds_per_interval(interval: str) -> int:
    seconds_per_unit = {
        "s": 1,
        "m": 60,
        "h": 60 * 60,
        "d": 24 * 60 * 60,
        "w": 7 * 24 * 60 * 60,
    }
    try:
        return int(interval[:-1]) * seconds_per_unit[interval[-1]] * 1000
    except (ValueError, KeyError):
        return None

data_selections = input("Enter the data selection (e.g., BTCUSDT_15m): ")

file_path = fr'D:\Personal_Document\GitHub\CNN-Bi-LSTM-Research\training-data\Model-3\{data_selections}_data.csv'

# Split the data_selections string into TICKET and interval
TICKET, interval_str = data_selections.rsplit('_', 1)

# Map the interval string to its corresponding KLINE_INTERVAL attribute
interval_map = {
    '1m': Client.KLINE_INTERVAL_1MINUTE,
    '3m': Client.KLINE_INTERVAL_3MINUTE,
    '5m': Client.KLINE_INTERVAL_5MINUTE,
    '15m': Client.KLINE_INTERVAL_15MINUTE,
    '30m': Client.KLINE_INTERVAL_30MINUTE,
    '1h': Client.KLINE_INTERVAL_1HOUR,
    '2h': Client.KLINE_INTERVAL_2HOUR,
    '4h': Client.KLINE_INTERVAL_4HOUR,
    '6h': Client.KLINE_INTERVAL_6HOUR,
    '8h': Client.KLINE_INTERVAL_8HOUR,
    '12h': Client.KLINE_INTERVAL_12HOUR,
    '1d': Client.KLINE_INTERVAL_1DAY,
    '3d': Client.KLINE_INTERVAL_3DAY,
    '1w': Client.KLINE_INTERVAL_1WEEK,
    '1M': Client.KLINE_INTERVAL_1MONTH,
}

KLINE_INTERVAL = interval_map.get(interval_str, None)
if KLINE_INTERVAL is None:
    print(f"Invalid interval: {interval_str}")
else:
    seconds = seconds_per_interval(KLINE_INTERVAL)
    if seconds is None:
        print(f"Cannot convert interval '{KLINE_INTERVAL}' to milliseconds.")

    
if os.path.exists(file_path):
    print("Found Request file")
    print(f"Requesting {TICKET} - {interval_str}")
    hist_df = pd.read_csv(file_path)
    print(f"Requesting {TICKET} - {interval_str} - Complete")
else:
    print(f"{TICKET} - {interval_str} data is not available")
    print(f"Attempting to retrieve {TICKET} - {interval_str}")
    historical = client.get_historical_klines(TICKET, KLINE_INTERVAL, '1 Jan 2019')
    print(f"Requesting Complete")
    print(f"Sorting Data columes...")
    hist_df = pd.DataFrame(historical)
    hist_df.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                        'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']
    
    hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')
    hist_df['Close Time'] = pd.to_datetime(hist_df['Close Time']/1000, unit='s')
    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']
    hist_df[numeric_columns] = hist_df[numeric_columns].apply(pd.to_numeric, axis=1)


hist_df.to_csv(file_path, index=False)
print("Preprocessing Data - Complete")

BTCUSDT - 2h data is not available
Attempting to retrieve BTCUSDT - 2h
Requesting Complete
Sorting Data columes...
Preprocessing Data - Complete
